In [56]:
import requests
from bs4 import BeautifulSoup
import json
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

all_links = []

for page in range(1, 402):
    url = f"https://www.coursera.org/directory/courses?page={page}"
    print(f"Đang xử lý trang {page}...")

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi tải trang {page}: {e}")
        continue

    soup = BeautifulSoup(response.text, "html.parser")
    columns = soup.find_all("div", {"data-testid": "linksColumn"})

    if not columns:
        print(f"Không tìm thấy dữ liệu ở trang {page}")
        continue

    for column in columns:
        links = column.find_all("a", href=True)
        for link in links:
            href = link["href"]
            if href.startswith("/learn/"):
                full_url = "https://www.coursera.org" + href
                all_links.append(full_url)

    time.sleep(1)  # Delay tránh bị chặn

# Lưu vào file JSON
with open("coursera_links.json", "w", encoding="utf-8") as f:
    json.dump(all_links, f, ensure_ascii=False, indent=2)

print(f"Đã lưu {len(all_links)} link vào coursera_links.json")


Đang xử lý trang 1...
Đang xử lý trang 2...
Đang xử lý trang 3...
Đang xử lý trang 4...
Đang xử lý trang 5...
Đang xử lý trang 6...
Đang xử lý trang 7...
Đang xử lý trang 8...
Đang xử lý trang 9...
Đang xử lý trang 10...
Đang xử lý trang 11...
Đang xử lý trang 12...
Đang xử lý trang 13...
Đang xử lý trang 14...
Đang xử lý trang 15...
Đang xử lý trang 16...
Đang xử lý trang 17...
Đang xử lý trang 18...
Đang xử lý trang 19...
Đang xử lý trang 20...
Đang xử lý trang 21...
Đang xử lý trang 22...
Đang xử lý trang 23...
Đang xử lý trang 24...
Đang xử lý trang 25...
Đang xử lý trang 26...
Đang xử lý trang 27...
Đang xử lý trang 28...
Đang xử lý trang 29...
Đang xử lý trang 30...
Đang xử lý trang 31...
Đang xử lý trang 32...
Đang xử lý trang 33...
Đang xử lý trang 34...
Đang xử lý trang 35...
Đang xử lý trang 36...
Đang xử lý trang 37...
Đang xử lý trang 38...
Đang xử lý trang 39...
Đang xử lý trang 40...
Đang xử lý trang 41...
Đang xử lý trang 42...
Đang xử lý trang 43...
Đang xử lý trang 44.

In [4]:
import requests
from bs4 import BeautifulSoup
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

lock = threading.Lock()  # Đảm bảo ghi file thread-safe

def get_soup_from_url(url, headers=None):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"Request error for {url}: {e}")
        return None

def extract_skills(soup):
    anchors = soup.find_all("a", attrs={"data-click-key": lambda x: x and "seo_skills_link_tag" in x})
    return [a.get_text(strip=True) for a in anchors]

def extract_what_you_will_learn(soup):
    sections = soup.find_all('section')
    if len(sections) > 1:
        viewers = sections[1].find_all("div", attrs={"data-testid": lambda x: x and "cml-viewer" in x})
        return [viewer.get_text(strip=True) for viewer in viewers[2:]]
    return []

def extract_title(soup):
    title = soup.find('h1', attrs={"class": lambda x: x and "cds-119 cds-Typography-base css-1xy8ceb cds-121" in x})
    return title.get_text(strip=True) if title else ""

def extract_course_description(soup):
    main = soup.find('main')
    if not main:
        return ""
    about_section = main.find("div", attrs={"class": lambda x: x and "rc-TogglableContent about-section" in x})
    return about_section.get_text(strip=True) if about_section else ""

def save_course_data(course_data, output_file):
    with lock:
        with open(output_file, "a", encoding="utf-8") as f:
            f.write(json.dumps(course_data, ensure_ascii=False) + "\n")

def process_course(url, output_file):
    soup = get_soup_from_url(url, headers={"User-Agent": "Mozilla/5.0"})
    if not soup:
        return

    course_data = {
        "title": extract_title(soup),
        "url": url,
        "skills": extract_skills(soup),
        "what_you_will_learn": extract_what_you_will_learn(soup),
        "description": extract_course_description(soup)
    }

    save_course_data(course_data, output_file)

def load_links(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

if __name__ == "__main__":
    input_file = "coursera_links.json"
    output_file = "coursera_courses_data.jsonl"  # JSON Lines format

    course_urls = load_links(input_file)

    max_threads = 10  # Tùy theo mạng + giới hạn Coursera
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = []
        for idx, url in enumerate(course_urls, 1):
            print(f"Đang lên lịch xử lý: {url}")
            futures.append(executor.submit(process_course, url, output_file))

        for future in as_completed(futures):
            pass  # có thể xử lý kết quả, báo lỗi ở đây nếu cần


Đang lên lịch xử lý: https://www.coursera.org/learn/serverless-computing-using-aws-lambda
Đang lên lịch xử lý: https://www.coursera.org/learn/accounting-for-ma-1
Đang lên lịch xử lý: https://www.coursera.org/learn/advanced-cybersecurity-techniques
Đang lên lịch xử lý: https://www.coursera.org/learn/advanced-strategies-for-sustainable-business
Đang lên lịch xử lý: https://www.coursera.org/learn/financial-engineering-advancedtopics
Đang lên lịch xử lý: https://www.coursera.org/learn/employeesguidetocybersecurity
Đang lên lịch xử lý: https://www.coursera.org/learn/applied-sustainability-engineering
Đang lên lịch xử lý: https://www.coursera.org/learn/applying-machine-learning-to-your-data-with-gc-fr
Đang lên lịch xử lý: https://www.coursera.org/learn/assist-public-sector-decision-makers-through-policy-analysis
Đang lên lịch xử lý: https://www.coursera.org/learn/mooc-atencion-inclusiva-al-publico-con-discapacidad
Đang lên lịch xử lý: https://www.coursera.org/learn/ictus-agudo-escala-race
Đa

In [5]:
import json

# Từ khóa liên quan đến IT (có thể mở rộng thêm nếu muốn)
IT_KEYWORDS = [
    # Lập trình & phần mềm
    "programming", "coding", "software", "software engineering", "app development",
    "mobile development", "android", "ios", "flutter", "react", "react native",
    "frontend", "backend", "full stack", "web development", "html", "css", "javascript",
    "typescript", "java", "python", "c++", "c#", "go", "kotlin", "php", "ruby",

    # Cơ sở dữ liệu & hệ thống
    "database", "sql", "mysql", "postgresql", "nosql", "mongodb", "firebase",
    "data engineering", "data warehouse", "etl", "big data", "hadoop", "spark",

    # Mạng & bảo mật
    "network", "networking", "cybersecurity", "security", "ethical hacking",
    "penetration testing", "firewall", "vpn", "linux security", "cloud security",

    # Điện toán đám mây
    "cloud", "aws", "azure", "gcp", "google cloud", "cloud computing", "devops",
    "docker", "kubernetes", "ci/cd", "infrastructure", "terraform",

    # Trí tuệ nhân tạo & dữ liệu
    "machine learning", "deep learning", "neural networks", "computer vision",
    "natural language processing", "artificial intelligence",
    "data science", "data analysis", "data analytics", "pandas", "numpy",
    "scikit-learn", "tensorflow", "pytorch",

    # CNTT nói chung
    "information technology", "computer science", "operating systems",
    "linux", "windows server", "system administration", "technical support",
    "it support", "help desk", "virtualization", "vmware", "bash", "shell scripting"
]


def is_it_course(course_data, keywords):
    text = (
        course_data.get("description", "").lower() + " " +
        " ".join(course_data.get("skills", [])).lower() + " " +
        " ".join(course_data.get("what_you_will_learn", [])).lower()
    )
    return any(kw in text for kw in keywords)

def load_courses(jsonl_file):
    with open(jsonl_file, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

def save_it_courses(it_courses, output_file):
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(it_courses, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    input_file = "coursera_courses_data.jsonl"
    output_file = "it_courses.json"

    all_courses = load_courses(input_file)
    it_courses = [course for course in all_courses if is_it_course(course, IT_KEYWORDS)]

    print(f"Đã lọc được {len(it_courses)} khóa học IT từ tổng số {len(all_courses)} khóa học.")
    save_it_courses(it_courses, output_file)


Đã lọc được 11347 khóa học IT từ tổng số 14017 khóa học.
